## Part 1: Preprocessing

In [65]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split


In [66]:


#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [67]:
# Determine the number of unique values in each column.
print("Columns with NaN values:", attrition_df.columns[attrition_df.isna().any()].tolist())
print ()
print (attrition_df.nunique())
print (attrition_df.dtypes)
print (attrition_df.shape)

Columns with NaN values: []

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64
Age                          int64
Attrition                   object
BusinessTravel              object
Department      

In [68]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
object_df = attrition_df.select_dtypes(include=['object'])

# Loop through each column in object_df and print the unique values
for column in object_df.columns:
    print(f"Unique values in '{column}': {object_df[column].unique()}")

Unique values in 'Attrition': ['Yes' 'No']
Unique values in 'BusinessTravel': ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']
Unique values in 'Department': ['Sales' 'Research & Development' 'Human Resources']
Unique values in 'EducationField': ['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']
Unique values in 'JobRole': ['Sales Executive' 'Research Scientist' 'Laboratory Technician'
 'Manufacturing Director' 'Healthcare Representative' 'Manager'
 'Sales Representative' 'Research Director' 'Human Resources']
Unique values in 'MaritalStatus': ['Single' 'Married' 'Divorced']
Unique values in 'OverTime': ['Yes' 'No']


In [69]:
def preprocess_ohe(df, ohe_column_list):
    # Create a copy of the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # List to store DataFrames of encoded columns
    encoded_dfs = []

    # Iterate over each column specified for one-hot encoding
    for column in ohe_column_list:
        # Initialize the OneHotEncoder
        ohe_encoder = OneHotEncoder(handle_unknown='error', sparse_output=False) # drop='first'
        
        # Fit and transform the data of the column
        ohe_encoded = ohe_encoder.fit_transform(df_copy[[column]])
        
        # Create a DataFrame from the encoded data with appropriate column names
        ohe_encoded_df = pd.DataFrame(ohe_encoded, columns=ohe_encoder.get_feature_names_out(input_features=[column]))
        
        # Append the newly encoded DataFrame to the list
        encoded_dfs.append(ohe_encoded_df)
        
        # Drop the original column from the copy of the DataFrame
        df_copy.drop(column, axis=1, inplace=True)
    
    # Concatenate all the encoded DataFrames with the modified original DataFrame
    df_copy = pd.concat([df_copy] + encoded_dfs, axis=1)
    
    return df_copy

In [70]:
def preprocess_yn(df, yn_columns):
    # Create a copy of the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # Replace 'Yes' with 1 and 'No' with 0 in specified columns
    replace_values = {'Yes': 1, 'No': 0}
    df_copy[yn_columns] = df_copy[yn_columns].replace(replace_values)
    
    return df_copy

def test_train_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    return X_train, X_test, y_train, y_test

In [71]:
# Encode Features

# make lists
ohe_feature = ['BusinessTravel', 'Department', 'EducationField','JobRole','MaritalStatus']
yn_columns  = [ 'Attrition', 'OverTime']

attrition_encoded_df1 = preprocess_ohe(attrition_df,ohe_feature)
attrition_encoded_df2  = preprocess_yn(attrition_encoded_df1,yn_columns)

print (attrition_encoded_df2)
print (attrition_encoded_df2.dtypes)

      Age  Attrition  DistanceFromHome  Education  EnvironmentSatisfaction  \
0      41          1                 1          2                        2   
1      49          0                 8          1                        3   
2      37          1                 2          2                        4   
3      33          0                 3          4                        4   
4      27          0                 2          1                        1   
...   ...        ...               ...        ...                      ...   
1465   36          0                23          2                        3   
1466   39          0                 6          1                        4   
1467   27          0                 4          3                        2   
1468   49          0                 2          3                        4   
1469   34          0                 8          3                        2   

      HourlyRate  JobInvolvement  JobLevel  JobSatisfaction  \


C:\Users\gefhz\AppData\Local\Temp\ipykernel_63376\2089520110.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy[yn_columns] = df_copy[yn_columns].replace(replace_values)


In [72]:
# Create y_df with the Attrition and Department columns
y_df = attrition_encoded_df2.filter(regex='^Depart|Attrition')
y_department = attrition_encoded_df2.filter(regex='^Depart')
y_attrition = attrition_encoded_df2.filter(regex='^Attrition')
print (y_df)
print (y_department)
print (y_attrition)

      Attrition  Department_Human Resources  \
0             1                         0.0   
1             0                         0.0   
2             1                         0.0   
3             0                         0.0   
4             0                         0.0   
...         ...                         ...   
1465          0                         0.0   
1466          0                         0.0   
1467          0                         0.0   
1468          0                         0.0   
1469          0                         0.0   

      Department_Research & Development  Department_Sales  
0                                   0.0               1.0  
1                                   1.0               0.0  
2                                   1.0               0.0  
3                                   1.0               0.0  
4                                   1.0               0.0  
...                                 ...               ...  
1465           

In [73]:
# Create a list of at least 10 column names to use as X data
all_columns = attrition_encoded_df2.columns.tolist()
y_columns = y_df.columns.tolist()
feature_columns = [col for col in all_columns if col not in y_columns]

# Create X_df using your selected columns

X = attrition_encoded_df2[feature_columns]

print (y_columns)
print (feature_columns)
print (X)
print (y_df)

['Attrition', 'Department_Human Resources', 'Department_Research & Development', 'Department_Sales']
['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'BusinessTravel_Non-Travel', 'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely', 'EducationField_Human Resources', 'EducationField_Life Sciences', 'EducationField_Marketing', 'EducationField_Medical', 'EducationField_Other', 'EducationField_Technical Degree', 'JobRole_Healthcare Representative', 'JobRole_Human Resources', 'JobRole_Laboratory Technician', 'JobRole_Manager', 'JobRole_Manufacturing Director', 'JobRole_Research Director', 'JobRole_Research Scientist', 'Jo

In [74]:
# Split the data into training and testing sets
X_train, X_test, y_attrition_train, y_attrition_test, y_department_train, y_department_test  =  train_test_split(X,y_attrition,y_department)

print (X_train.columns)
print (y_attrition_train.columns)
print (y_department_train.columns)

Index(['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction',
       'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'BusinessTravel_Non-Travel',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'EducationField_Human Resources', 'EducationField_Life Sciences',
       'EducationField_Marketing', 'EducationField_Medical',
       'EducationField_Other', 'EducationField_Technical Degree',
       'JobRole_Healthcare Representative', 'JobRole_Human Resources',
       'JobRole_Laboratory Technician', 'JobRole_Manager',
       'JobRole_Manufacturing Director', 'JobRole_Research Director',
       'JobRole_Research Scientist', 

In [75]:
# Step 1: Create a StandardScaler
scaler = StandardScaler()

# Step 2: Fit the StandardScaler to the training data
scaler.fit(X_train)

# Step 3: Transform the features training dataset
X_train_scaled = scaler.transform(X_train)

# Step 4: Transform the features testing dataset
X_test_scaled = scaler.transform(X_test)


In [76]:
# done earlier
#
# Create a OneHotEncoder for the Department column
# Fit the encoder to the training data
# Create two new variables by applying the encoder
# to the training and testing data

In [77]:
# done earlier
# 
# # Create a OneHotEncoder for the Attrition column
# Fit the encoder to the training data
# Create two new variables by applying the encoder
# to the training and testing data

## Create, Compile, and Train the Model

In [78]:
# Find the number of columns in the X training data
input_nodes  = X_train_scaled.shape[1]

num_nodes_first_hidden_layer = ((input_nodes * 50)  )

# Define the number of hidden nodes for the second hidden layer

# Create the input layer
input_layer = layers.Input(shape=(input_nodes,), name='input_features')


# Create at least two shared layers

# Shared hidden layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

# Define the number of hidden nodes for the second hidden layer

# Define the number of neurons in the output layer

num_output_neurons = 1 

In [79]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer

department_output = layers.Dense(3, activation='softmax', name='department_output')(shared_layer2)

In [80]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(shared_layer2)


In [81]:
# Create the model
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_features (InputLayer)    [(None, 42)]         0           []                               
                                                                                                  
 dense_6 (Dense)                (None, 64)           2752        ['input_features[0][0]']         
                                                                                                  
 dense_7 (Dense)                (None, 32)           2080        ['dense_6[0][0]']                
                                                                                                  
 department_output (Dense)      (None, 3)            99          ['dense_7[0][0]']                
                                                                                            

In [85]:
# Train the model

# Fit the model
model.fit(
    X_train,
    {'department_output': y_department_train, 'attrition_output':y_attrition_train},
    epochs=100,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/100
28/28 [==============================] - 0s 6ms/step - loss: 0.3167 - department_output_loss: 0.0369 - attrition_output_loss: 0.2798 - department_output_accuracy: 0.9898 - attrition_output_accuracy: 0.8956 - val_loss: 0.4732 - val_department_output_loss: 0.0964 - val_attrition_output_loss: 0.3768 - val_department_output_accuracy: 0.9774 - val_attrition_output_accuracy: 0.8597
Epoch 2/100
28/28 [==============================] - 0s 3ms/step - loss: 0.3106 - department_output_loss: 0.0308 - attrition_output_loss: 0.2798 - department_output_accuracy: 0.9932 - attrition_output_accuracy: 0.8899 - val_loss: 0.4596 - val_department_output_loss: 0.0857 - val_attrition_output_loss: 0.3738 - val_department_output_accuracy: 0.9774 - val_attrition_output_accuracy: 0.8778
Epoch 3/100
28/28 [==============================] - 0s 3ms/step - loss: 0.3174 - department_output_loss: 0.0375 - attrition_output_loss: 0.2799 - department_output_accuracy: 0.9886 - attrition_output_accuracy: 0.8944 

In [ ]:
# Evaluate the model with the testing data


In [ ]:
# Print the accuracy for both department and attrition


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 